In [ ]:
# https://www.geeksforgeeks.org/deploy-machine-learning-model-using-flask/#:~:text=%23%20prediction%20function,prediction%20%3D%20prediction)
# importing libraries
import warnings
warnings.filterwarnings("ignore")

# importing libraries
import datetime
from flask import Flask, jsonify, request
from flask import Flask, render_template
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tqdm import tqdm
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.metrics import accuracy_score
import flask
app = Flask(__name__)


def preprocessing(data):
    stop_words = stopwords.words('english')

    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

        # https://gist.github.com/sebleier/554280

        # Remove Emoji
    def deEmojify(text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def preprocess_text(text_data):
        preprocessed_text = []
        # tqdm is for printing the status bar
        for sentance in tqdm(text_data):
            sent = decontracted(sentance)
            sent=deEmojify(sent)
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\n', ' ')
            sent = sent.replace('\\"', ' ')
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
            # https://gist.github.com/sebleier/554280
            sent = ' '.join(e for e in sent.split() if e not in stop_words)
            preprocessed_text.append(sent.lower().strip())
        return preprocessed_text
    
    data['comment'] = preprocess_text(data['comment'].values)
    data['parent_comment'] = preprocess_text(data['parent_comment'].values)
        
    data['subreddit'] = data['subreddit'].str.replace(' ','')
    data['subreddit'] = data['subreddit'].str.replace('-','_')
    data['subreddit'] = data['subreddit'].str.lower()
    
    data['author'] = data['author'].str.replace(' ','')
    data['author'] = data['author'].str.replace('-','_')
    data['author'] = data['author'].str.lower()
    
    neg_comment = []
    neu_comment = []
    pos_comment = []
    comp_comment = []

    sid = SentimentIntensityAnalyzer()

    for sentence in tqdm(data['comment']):
        ss = sid.polarity_scores(sentence)
        neg_comment.append(ss['neg'])
        neu_comment.append(ss['neu'])
        pos_comment.append(ss['pos'])
        comp_comment.append(ss['compound'])

    data['Neg_comment'] = neg_comment
    data['Neu_comment'] = neu_comment
    data['Pos_comment'] = pos_comment
    data['Compound_comment'] = comp_comment
    
    neg_pcomment = []
    neu_pcomment = []
    pos_pcomment = []
    comp_pcomment = []

    sid = SentimentIntensityAnalyzer()

    for sentence in tqdm(data['parent_comment']):
        ss = sid.polarity_scores(sentence)
        neg_pcomment.append(ss['neg'])
        neu_pcomment.append(ss['neu'])
        pos_pcomment.append(ss['pos'])
        comp_pcomment.append(ss['compound'])
    
    data['Neg_pcomment'] = neg_pcomment
    data['Neu_pcomment'] = neu_pcomment
    data['Pos_pcomment'] = pos_pcomment
    data['Compound_pcomment'] = comp_pcomment
    
    data['comment_len']=data['comment'].apply(lambda x:len(x.split()))
    data['pcomment_len']=data['parent_comment'].apply(lambda x:len(x.split()))
    
    cat_author, cat_subreddit = pickle.load(open("cat.pkl", 'rb'))
    data['author'] = cat_author.transform(data['author'])
    data['subreddit'] = cat_subreddit.transform(data['subreddit'])
    
    scaler_score, scaler_ups, scaler_negc, scaler_neuc, scaler_posc, scaler_compc, scaler_negpc, scaler_neupc, scaler_pospc, scaler_comppc, scaler_comlen, scaler_pcomlen = pickle.load(open("scale.pkl", 'rb'))
    data['score'] = scaler_score.transform(data['score'].values.reshape(-1,1))    
    data['ups'] = scaler_ups.transform(data['ups'].values.reshape(-1,1))
    data['Neg_comment'] = scaler_negc.transform(data['Neg_comment'].values.reshape(-1,1))
    data['Neu_comment'] = scaler_neuc.transform(data['Neu_comment'].values.reshape(-1,1))
    data['Pos_comment'] = scaler_posc.transform(data['Pos_comment'].values.reshape(-1,1))
    data['Compound_comment'] = scaler_compc.transform(data['Compound_comment'].values.reshape(-1,1))
    data['Neg_pcomment'] = scaler_negpc.transform(data['Neg_pcomment'].values.reshape(-1,1))
    data['Neu_pcomment'] = scaler_neupc.transform(data['Neu_pcomment'].values.reshape(-1,1))
    data['Pos_pcomment'] = scaler_pospc.transform(data['Pos_pcomment'].values.reshape(-1,1))
    data['Compound_pcomment'] = scaler_comppc.transform(data['Compound_pcomment'].values.reshape(-1,1))
    data['comment_len'] = scaler_comlen.transform(data['comment_len'].values.reshape(-1,1))
    data['pcomment_len'] = scaler_pcomlen.transform(data['pcomment_len'].values.reshape(-1,1))
    
    text_features = data[['comment', 'parent_comment']]
    data.drop(['comment', 'parent_comment'], axis=1, inplace=True)
    
    num_features = data.values
    
    token_comment, token_pcomment =  pickle.load(open('tokens.pkl','rb'))
    sequence_com = token_comment.texts_to_sequences(text_features['comment'])
    pad_comment = pad_sequences(sequence_com, maxlen=50, padding='post')
    sequence_pcom = token_pcomment.texts_to_sequences(text_features['parent_comment'])
    pad_pcomment = pad_sequences(sequence_pcom, maxlen=500, padding='post')    
    
    return pad_comment, pad_pcomment, num_features

model = keras.models.load_model('final_model.h5')


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():        
    df = pd.DataFrame()
    df['comment'] = list([str(request.form['comment'])])
    df['author'] = list([str(request.form['author'])])
    df['subreddit'] = list([str(request.form['subreddit'])])
    df['score'] = list([int(request.form['score'])]) 
    df['ups'] = list([int(request.form['ups'])])
    df['downs'] = list([int(request.form['downs'])])
    df['parent_comment'] = list([str(request.form['parent_comment'])]) 
    
    print(df)
        
    final_data = preprocessing(df)    
    
    y_pred = model.predict(final_data)
    
    if y_pred >= 0.5:
        prediction = 'Sarcastic'
    else:
        prediction = 'Non-Sarcastic'
    return render_template("index.html", prediction_text = 'Comment on the post is {}'.format(prediction))


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\80316\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jun/2022 11:50:50] "GET / HTTP/1.1" 200 -


                            comment       author        subreddit  score  ups  \
0  you should file a dmca complaint  BrokenStool  GlobalOffensive      1    1   

   downs                                     parent_comment  
0      0  This skin looks exactly like the SA Highwayman...  


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1003.66it/s]


1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [24/Jun/2022 11:51:36] "POST /predict HTTP/1.1" 200 -
